<span style="font-size: 5em">🦜</span>

# __LangGraph Essentials__
## Lab 1: States & Nodes

<img src="../assets/States_Nodes.png" align="left" width="600" style="margin-right:15px;"/>


LangGraph organizes workflows as graphs where nodes are functions and edges define execution flow. All nodes share a common state that gets passed between them. This notebook shows how to define state, create nodes, and connect them into an executable graph.


In [16]:
from IPython.display import Image, display
import operator
from typing import Annotated, List, Literal, TypedDict
from langgraph.graph import END, START, StateGraph
from langgraph.types import Command, interrupt

<a id='state_definition'></a>

In [17]:
class State(TypedDict):
    nlist: List[str]
    name_from: str

<a id='node_function'></a>


In [18]:
def node_a(state: State) -> State:
    print(f"node a is receiving {state['nlist']}")
    note = f"Update from node_a: Hello World from Node a; from {state['name_from']}!"
    return(State(nlist = [note], name_from=state['name_from']))


<a id='graph_building'></a>

In [19]:
def node_b(state: State) -> State:
    print(f"node b is receiving {state['nlist']}")
    note = f"Update from node_b: Check weather example"
    source_from = "Weather API"
    return(State(nlist = [note], name_from=source_from))

In [ ]:
builder = StateGraph(State)
builder.add_node("node_a", node_a)
builder.add_node("node_b", node_b)
builder.add_edge(START, "node_a")
builder.add_edge("node_a", "node_b")
builder.add_edge("node_b", END)
graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))
# print(graph.get_graph().draw_mermaid())

<a id='graph_invoke'></a>

In [31]:
initial_state = State(
    nlist = ["Hello Node a, how are you?"],
    name_from="Val Neyman"
)
graph.invoke(initial_state)

node a is receiving ['Hello Node a, how are you?']
node b is receiving ['Update from node_a: Hello World from Node a; from Val Neyman!']


{'nlist': ['Update from node_b: Check weather example'],
 'name_from': 'Weather API'}

## Takeaways

Setup:

- State: All nodes share the same [state](#state_definition) which can be a Python TypedDict, dataclass, or a Pydantic BaseModel
- Nodes: Defined as simple [Python functions](#node_function) that receive state as input and return updated state

Execution (invoke):

- Runtime: When you call [invoke](#graph_invoke), the graph initializes the input state from your invoke statement and determines which nodes to run
- State Flow: Each node receives the current state as input, executes its logic, and returns an updated state
- Graph Return: After all nodes complete execution, the graph returns the final state value

Try Next:

- Add another node to the [graph](#graph_building) and connect it with edges
- Modify the print statement in the [node function](#node_function) or change the initial state message
- Extend the [State TypedDict](#state_definition) with additional fields to pass more data between nodes